# overdose death

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import re

In [2]:
# read in US death statistics
original_zip = ZipFile('../00_source/US_VitalStatistics.zip', 'r')
new_zip = ZipFile('new_archve.zip', 'w')
for item in original_zip.infolist():
    buffer = original_zip.read(item.filename)
    if not str(item.filename).startswith('__MACOSX/'):
        new_zip.writestr(item, buffer)
new_zip.close()
original_zip.close()

In [3]:
new_zip = ZipFile('new_archve.zip', 'r')
dfs = {}

for text_file in new_zip.infolist():
    dfs[re.search('2\d\d\d', text_file.filename).group(0)] = pd.read_csv(new_zip.open(text_file.filename), sep = "\t", usecols = [1, 2, 3, 5, 7])[:-15]
    pass
new_zip.close()
!rm new_archve.zip

origin = pd.DataFrame({})
for key in dfs.keys():
    origin = origin.append(dfs[key])
origin.drop('County Code', axis = 1, inplace = True)

In [4]:
origin['Drug/Alcohol Induced Cause'].unique()

array(['All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Unintentional (X40-X44)',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Drug poisonings (overdose) Suicide (X60-X64)',
       'All other drug-induced causes',
       'All other alcohol-induced causes',
       'Alcohol poisonings (overdose) (X45, X65, Y15)',
       'Drug poisonings (overdose) Homicide (X85)'], dtype=object)

In [5]:
# check whihc county has missing value
origin[origin['Deaths'] == 'Missing']['County'].unique()
# index_names = origin[origin['Deaths'] == 'Missing'].index
# origin = origin.drop(index_names)

# replace missing value with 10
origin['Deaths'] = origin['Deaths'].replace('Missing', 10)

In [6]:
origin['Deaths'] = origin['Deaths'].astype('int64')
origin['Year'] = origin['Year'].astype('int64')
origin[['County','State']] = origin.County.str.split(", ",expand=True,)

In [7]:
# convert abbreviation to full name
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
origin['State'] = origin['State'].map(states)

In [8]:
names = []
for name in origin['Drug/Alcohol Induced Cause'].unique():
    if re.match('Drug poisonings.*', name):
        names.append(name)
        pass
    pass

interDose = origin[origin['Drug/Alcohol Induced Cause'].isin(names)].copy()

In [9]:
finalDose = interDose.groupby(['State', 'County', 'Year'], as_index = False).sum()[['State', 'County', 'Year', 'Deaths']].rename({'Deaths':'TotalOverdose'}, axis = 'columns').copy()

# pop

In [11]:
pop = pd.read_csv('../20_intermediate_files/FinalPopDataset.csv')

In [12]:
# subset population dataset
states = finalDose.State.unique().copy()
subPop = pop[pop['STATE'].isin(states) & (pop['YEAR'] >= 2003) & (pop['YEAR'] <= 2015)].reset_index(drop = True).copy()
index_names = subPop[subPop['COUNTY'].isin(states)].index
subPop = subPop.drop(index_names)

# merging overdose and pop

In [15]:
final = pd.merge(subPop, finalDose, how = 'left', left_on = ['STATE', 'COUNTY', 'YEAR'], right_on = ['State', 'County', 'Year'], indicator = True)
final.value_counts('_merge')

_merge
left_only     32995
both           7838
right_only        0
dtype: int64

In [16]:
final.drop(['County', 'Year', 'State'], axis = 1, inplace = True)
final['TotalOverdose'] = final['TotalOverdose'].fillna(0)

In [18]:
final.loc[final.TotalOverdose == 0, "TotalOverdose" ] = 10
final.loc[final.COUNTY == 'Loving County', "TotalOverdose" ] = 1
final.loc[final.COUNTY == 'King County', "TotalOverdose" ] = 1
final.loc[final.POP < 1000, "TotalOverdose" ] = 1

# calculate overdose death

In [20]:
final.drop('_merge', axis = 1, inplace = True)

In [21]:
final['OverdoseProp'] = final['TotalOverdose'] / final['POP']

In [22]:
final['PolicyState'] = (final['STATE'] == 'Florida') | ((final['STATE'] == 'Texas')) | (final['STATE'] == 'Washington')

In [23]:
nearFL = ['Florida', 'Louisiana', 'Mississippi', 'South Carolina']
nearTX = ['Texas', 'Arkansas', 'New Mexico', 'Kansas']
nearWA = ['Washington', 'Colorado', 'Oregon', 'California']

In [24]:
final['Post'] = ((final['STATE'].isin(nearFL)) & (final['YEAR'] >= 2010)) | ((final['STATE'].isin(nearTX)) & (final['YEAR'] >= 2007)) | ((final['STATE'].isin(nearWA)) & (final['YEAR'] >= 2012))

In [25]:
len(final[final['STATE'] == 'Texas']['COUNTY'].unique())

254

In [26]:
final.to_csv('../20_intermediate_files/state_county_death.csv', index = False)

# Loop before simplification

In [12]:
#deathTypes = origin['Drug/Alcohol Induced Cause'].unique().copy()
#years = final.YEAR.unique().copy()

#years = sorted(origin['Year'].unique()).copy()
#final = final[final['STATE'].isin(states) & (final['YEAR'] >= 2003) & (final['YEAR'] <= 2015)].reset_index(drop = True).copy()

#state_county = {}

#for state in states:
#    counties = final[final['STATE'] == state]['COUNTY'].unique().copy()
#    state_county[state] = counties
#    pass

In [16]:
#for state, counties in state_county.items():
#    for county in counties: 
#        for year in years:
#            window = final[(final['STATE'] == state) & (final['COUNTY'] == county) & (final['YEAR'] == year)].copy()
#            population = window['POP'].iloc[0]
#            for death in deathTypes:
#                existingDeath = window['Drug/Alcohol Induced Cause'].unique()
#                if death not in existingDeath:
#                    new_row = {'STATE':state, 'COUNTY':county, 'YEAR':year, 'POP':population, 'Drug/Alcohol Induced Cause':death, 'Deaths':0, '_merge':'Missing'}
#                    final = final.append(new_row, ignore_index=True).copy()

In [17]:
#sub = final.copy()
#index_names = sub[sub['_merge'] == 'left_only'].index
#sub = sub.drop(index_names)
#sub.value_counts('_merge')
#sub.drop('_merge', axis = 1, inplace = True)